In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib

#load dataset
df = pd.read_csv("hairstyle_prediction_dataset_1000.csv")

#features and target
X = df[['face_width','jaw_width', 'forehead_width', 'chin_length']]
y = df['face_shape']

#Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =42)

#Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#GridSearchCV: Hyperparameter tuning

param_grid ={
    'C': [0.1, 1, 10],
    'gamma':['scale', 0.01, 0.001],
    'kernel':['rbf']
}

grid_search = GridSearchCV(
    SVC(), param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)

#Best model from grid search
svm_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


#Evaluate
y_pred = svm_model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#-------Rule-based hairstyle recommendation system--------#

face_to_hairstyle = df.drop_duplicates(subset=['face_shape'])[['face_shape','best_hairstyle']].set_index('face_shape')['best_hairstyle'].to_dict()

def predict_face_shape_and_hairstyle(input_features):
    #Create DataFrame with matching column names
    input_df = pd.DataFrame([input_features], columns=['face_width', 'jaw_width', 'forehead_width', 'chin_length'])

    #Scale and predict
    scaled_input = scaler.transform(input_df)
    predicted_shape = svm_model.predict(scaled_input)[0]

    #Recommend hairstyle
    recommended_hairstyle = face_to_hairstyle.get(predicted_shape,"No suggestions")
    return predicted_shape,recommended_hairstyle

#Example
new_face =[150, 130, 153, 50] # face_width, jaw_width, forehead_width, chin_length
shape, hairstyle = predict_face_shape_and_hairstyle(new_face)
print(f"Predicted Face Shape: {shape}")
print(f"Recommended Hairstyle: {hairstyle}")




Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.935
              precision    recall  f1-score   support

       heart       0.90      0.84      0.87        32
      oblong       0.93      0.95      0.94        43
        oval       0.94      0.92      0.93        37
       round       0.89      0.93      0.91        43
      square       1.00      1.00      1.00        45

    accuracy                           0.94       200
   macro avg       0.93      0.93      0.93       200
weighted avg       0.94      0.94      0.93       200

Predicted Face Shape: oval
Recommended Hairstyle: straight_cut
